In [1]:
from edgeworth_env import *

2024-05-06 20:44:54,052	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
env = OligopolyMarket()

In [ ]:
env.reset()

({'firm_0': array([0., 0., 0., 0., 0.], dtype=float32),
  'firm_1': array([0., 0., 0., 0., 0.], dtype=float32),
  'firm_2': array([0., 0., 0., 0., 0.], dtype=float32),
  'firm_3': array([0., 0., 0., 0., 0.], dtype=float32),
  'firm_4': array([0., 0., 0., 0., 0.], dtype=float32)},
 {})

In [4]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

In [5]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [6]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-05-06 20:45:01,085	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [7]:
agent_ids = OligopolyMarket()._agent_ids
sym_policies = {agent_id: f"policy_firm_0" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = asym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)

config = (
        PPOConfig()
        .environment(OligopolyMarket)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 12000}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )


tuner.fit()

2024-05-06 20:45:02,184	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1314609) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1314609) 2024-05-06 20:45:07,096	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 640.0, 'num_env_steps_trained': 1024.0, 'total_loss': 48.985118516286214}, 'policy_firm_1': {'total_loss': 48.985118516286214, 'policy_loss': -0.037399948345652474, 'vf_loss': 9.970809721946717, 'vf_loss_unclipped': 173782426.86666667, 'vf_explained_var': 3.818422555923462e-06, 'entropy': 1.764908782641093, 'mean_kl_loss': 0.015493038985853975, 'default_optimizer_lr': 5.0000000000000016e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.45000001788139343}, 'policy_firm_3': {'total_loss': 9.590001539389293, 'policy_loss': -0.03506547214152912, 'vf_loss': 9.619214983781179, 'vf_loss_unclipped': 55957960.9625, 'vf_explained_var': 1.7171353101730345e-05, 'entropy': 2.511019679903984, 'mean_kl_loss': 0.01300442052828051, 'default_optimizer_lr': 5.0000000000000016e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.45000